# <center>Curricular Analytics Applied to Transfer Articulation: A Case Study</center>

<center>
    <b>Hayden W. Free$^\ddagger$, Gregory L. Heileman,$^\star$ and Chaouki T. Abdallah$^\bullet$</b> <br><br>
    $^\star$Department of Electrical & Computer Engineering <br>
    University of Arizona <br>
    heileman@arizona.edu <br><br>
    $^\ddagger$Department of Computer Science <br>
    University of Kentucky <br>
    hayden.free@uky.edu<br><br>
    $^\bullet$Department of Electrical & Computer Engineering <br>
    Georgia Institute of Technology <br>
    ctabdallah@gatech.edu
</center>

The Curricular Analytics Toolbox includes capablities that allow you to create customized degree plans according to various user-specifed optimization criteria. In order to use these capabilities, you first must have permission to access the CurricularOptimziation.jl package through a professional-level account. To obtain a professional-level account, please visit: http://CurricularAnalytics.org/pro. 

After obtaining your professional account, in order to use the CurricularOptimization tools, first enter package mode from the Julia REPL by pressing `]`, then enter: 

`pkg> add https://github.com/CurricularAnalytics/CurricularOptimization.jl.git`

The curricular optimization features make use of the [JuMP](https://github.com/JuliaOpt/JuMP.jl) domain-specific language for specifying optimization problems in Julia, and call the [Gurobi](https://www.gurobi.com) solver in order to solve the optimzaton problems. In order to use these features you must also install the solver, called [Gurobi Optimizer](https://www.gurobi.com/downloads/gurobi-optimizer-eula). Gurobi is a commercial product, and requires a license key; however, [academic licenses](https://www.gurobi.com/downloads/end-user-license-agreement-academic) are available at no cost.

After installing the Gurobi Solver on your computer, execute the following commands to use the optimization capabilities within the Curricular Analytics Toolbox: 

In [2]:
using CurricularAnalytics, CurricularVisualization, CurricularOptimization;

# Introduction

In many cases when a student transfers the credits from the original institution are not lost, but are either wasted or ineffecient in progressing towards the target degree.

### What makes this problem hard?
NOTE: automotive always trips people up, use that example. military credits **rarely** apply even though they're accepted
<img src="hard-problem.png" width="600" style="float:left">

### We want credits that apply, not just count!

# Example - Two Students Attempt to Study Basket Weaving
In the following example we demonstrate two students that take very similiar courses, but one student transfers from a community college. This example highlights the large difference in degree progress that can occur depending on how courses transfer.

For the sake of conserving space most of the input to perform these steps is contained in the `basket-weaving-example.jl` file, which is included with this notebook. Snippets of the various actions are highlighted below.

```julia
# Create stores for institutions courses and catalog.
college = "Tri-County Community College"
univ = "Big State U"
tri_county_catalog = CourseCatalog("2019-20 Academic Year", college)
big_state_catalog = CourseCatalog("2019-20 Academic Year", univ)
tri_county_courses = Array{Course,1}()
big_state_courses = Array{Course,1}()

# Create the tri county Courses 
push!(tri_county_courses, Course("Probability & Statistics", 3, prefix="MA", num="107", institution=college))
...
# Add tri county courses to catalog
add_course!(tri_county_catalog, tri_county_courses)


# Create the big state Courses
push!(big_state_courses, Course("Statistics", 3, prefix="MATH", num="120", institution=univ))
...
# Add big state courses to catalog
add_course!(big_state_catalog, big_state_courses)


# Big State U General Education Requirement
# Math Requirement
math = CourseSet("Math Requirement", 3, [course(big_state_catalog, "MATH", "120", "Statistics") => grade("D"),course(big_state_catalog, "MATH", "180", "Calculus I") => grade("D"), course(big_state_catalog, "MATH", "140", "College Trigonometry") => grade("D")], double_count = true)
#                                               ^^^ Double Count is critical here
...

# Construct full GenEd requirement set
gen_ed = RequirementSet("General Education Core", 9, convert(Array{AbstractRequirement,1}, [math, phys_ed, humanities]), description="General Education Requirements")


# Create BW Degree Requirements
bw_dr = RequirementSet("Degree Requirements", 30, convert(Array{AbstractRequirement,1}, [gen_ed, bw_major]), description="Degree Requirements for the BS Computer Science Degree")


# Create the transfer articulation map between tri county and big state
BSU_xfer = TransferArticulation("Big State U Transfer Articulation", univ, big_state_catalog)
add_transfer_catalog(BSU_xfer, tri_county_catalog)
add_transfer_course(BSU_xfer, [course_id("MATH", "120", "Statistics", univ)], tri_county_catalog.id, course_id("MA", "107", "Probability & Statistics", college))
...

# Create a transcript involving tri county coursework (Our transfer student)
tri_county_transcript = Array{CourseRecord,1}()
push!(tri_county_transcript, CourseRecord(course(tri_county_catalog, "MA", "107", "Probability & Statistics"), grade("A")))
...

# Map tri state courses and grades to big state courses and grades through transfer equivalences
BSU_equiv_transcript = Array{CourseRecord,1}()
for cr in tri_county_transcript
    ...
end


# Determine the requirements satisfied at BSU by the articulated courses
model = assign_courses(BSU_equiv_transcript, bw_dr, [applied_credits, requirement_level])
x = model.obj_dict[:x]
is_satisfied = satisfied(BSU_equiv_transcript, bw_dr, JuMP.value.(x))
show_requirements(bw_dr, satisfied=is_satisfied)
```

In [8]:
big_state_transcript = Array{CourseRecord,1}()
push!(big_state_transcript, CourseRecord(course(big_state_catalog, "MATH", "110", "College Algebra"), grade("A")))
push!(big_state_transcript, CourseRecord(course(big_state_catalog, "PHS", "110", "Swimming"), grade("B")))
push!(big_state_transcript, CourseRecord(course(big_state_catalog, "BW", "101", "Basic Basket Forms"), grade("A")))
push!(big_state_transcript, CourseRecord(course(big_state_catalog, "PHIL", "100", "Intro. Philosophy"), grade("D")))
push!(big_state_transcript, CourseRecord(course(big_state_catalog, "MATH", "140", "College Trigonometry"), grade("B")))

bsu_model = assign_courses(big_state_transcript, bw_dr, [applied_credits, requirement_level]);
x = bsu_model.obj_dict[:x];
bsu_satisfied = satisfied(big_state_transcript, bw_dr, JuMP.value.(x));
show_requirements(bw_dr, satisfied=bsu_satisfied)

LoadError: UndefVarError: big_state_catalog not defined

In [7]:
bsu_audit = audit_transcript(big_state_transcript, bw_dr, value.(x))

for (course,reason) in bsu_audit
    println("$(course.course.name) ---> $(reason)")
end

LoadError: UndefVarError: value not defined

# How These Students Compare

## Big State University Student
<img src="bsu-student.png" width="600">

<br>
<br>
<br>
<br>

## Tri County Transfer Student
<img src="tri-county-student.png" width="600">